# CFT – ECG Analysis

## Setup and Helper Functions

In [1]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs

from cft_analysis.datasets import CftDatasetProcessed

from IPython.display import Markdown

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

pg.options['round'] = 4

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

## Import Data

In [3]:
# get path to analysis results
base_path = Path("../../data")

In [4]:
results_path = base_path.joinpath("../results")
stats_path = results_path.joinpath("statistics")
tex_path = stats_path.joinpath("tex_tables")
bp.utils.file_handling.mkdirs([results_path, stats_path, tex_path])

paper_path = Path("../paper_path.json")
paper_tex_path = None
if paper_path.exists():
    paper_path = Path(json.load(paper_path.open(encoding="utf-8"))["paper_path"])
    paper_tex_path = paper_path.joinpath("tab")
    bp.utils.file_handling.mkdirs([paper_tex_path])

In [5]:
dataset = CftDatasetProcessed(base_path, exclude_subjects=True)
dataset

,condition,subject,phase,subphase
0,CFT,Vp01,MIST1,AT
1,CFT,Vp01,MIST1,AT
2,CFT,Vp01,MIST1,AT
3,CFT,Vp01,MIST1,AT
4,CFT,Vp01,MIST1,AT
...,...,...,...,...
4443,Control,Vp33,Post,Total
4444,Control,Vp33,Pre,Total
4445,Control,Vp33,Pre,Total
4446,Control,Vp33,Pre,Total


In [6]:
hue_order = ["Control", "CFT"]

## Response to the MIST

### Descriptive Analysis

To show the general efficacy of the MIST we compute the mean and maximum heart rate increase for each subject and MIST phase, respectively.  
Afterwards, we compute mean and standard deviation over all subjects:

In [7]:
hr_ensemble = dataset.heart_rate_ensemble
hr_ensemble.head()

subject          Vp01      Vp02       Vp04      Vp05       Vp15       Vp16  \
phase time                                                                   
MIST1 1     -8.317398 -0.412402   7.184292 -7.185016   9.180447 -15.601029   
      2     -3.720180 -4.127014  10.041075 -6.109130  14.538993 -13.528410   
      3     -5.866453 -8.723869   2.170225 -8.075388   4.105569 -19.610884   
      4     -9.985982 -8.748880  -3.774143 -3.142681  -7.994921 -23.350802   
      5    -11.751702 -3.616214 -12.368065 -9.587547  -9.542040 -24.727157   

subject          Vp18      Vp20      Vp25       Vp26  ...       Vp09  \
phase time                                            ...              
MIST1 1      2.743854 -8.664389  4.683156  -3.357802  ...  -7.818134   
      2      5.956126 -6.339915  2.560471  -4.909431  ...  -7.413341   
      3     11.966072 -4.329427  3.123288  -9.122872  ...  -2.236057   
      4      1.386065 -3.384193 -0.288092 -11.918682  ... -10.937985   
      5     -0.038145 -0.996683  5.123615 -12.620306  ...  -7.117339   

subject         Vp11      Vp17      Vp19       Vp21      Vp24      Vp30  \
phase time                                                                
MIST1 1     2.724037  1.743075  2.010397   3.328623 -7.181512  2.434562   
      2     3.339442 -0.058710 -0.131599   1.704383 -7.958105  3.777146   
      3     5.633895 -2.632612 -9.614859  19.556275 -7.720858 -2.844013   
      4     4.866643 -3.074540  1.220778  35.388658 -1.614905 -4.780654   
      5     2.432493  0.329262  1.085641  35.895015 -5.849688 -3.909095   

subject          Vp31       Vp32      Vp33  
phase time                                  
MIST1 1    -11.582174  14.173981  2.532088  
      2    -12.787637  17.518175  6.446413  
      3    -10.203479   8.834447 -1.229008  
      4     -4.495005   6.735395 -2.398628  
      5     -7.902182   5.209966  1.205645  

[5 rows x 25 columns]

In [8]:
hr_increase = hr_ensemble.groupby("phase").agg(["mean", "max"])
hr_increase = hr_increase.stack().agg(["mean", "std"], axis=1)
hr_increase.round(2)

mean    std
phase                   
MIST1 mean  -1.98   5.38
      max   26.00  10.50
MIST2 mean   3.85  10.54
      max   32.90  15.39
MIST3 mean  10.32  12.85
      max   39.02  17.13

### Statistical Analysis

**Population**: Control condition

**Analysis** (per measure):
1. *Increase BL-AT*: Check whether Arithmetic Tasks of the MIST causes significant HR(V) responses.
    * Procedure: Paired t-tests between subphases *BL* and *AT* for each MIST phase
    * Expected Result: Significant HR(V) responses for each MIST phase
2. *Increase over MIST*: Check whether the HR response increases over time (i.e., over MIST phases). 
    * Procedure: Repeated-measures ANOVA for each subphase
    * *(only for HR)*: Post-hoc test to check which pairs of MIST phases show significant differences
    * Expected Result: Significant main effect *MIST Phase* for each subphase


**Findings**: 
* Each MIST Phase induces stress, indicated by significant differences between $BL_{Loc}$ and $AT$ in each individual MIST Phase
* Stress levels increase over time, indicated by significant main effect *MIST Phase* and significant differences between MIST phases, especially between *MIST1* and *MIST3*

#### Increase BL-AT

In [9]:
display(Markdown("##### Prepare Data"))

condition = "Control"
subphases = ["BL", "AT"]
measures = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]

hr_analysis = dataset.get_subset(condition=condition, subphase=subphases).hr_hrv
hr_analysis = multi_xs(hr_analysis, measures, level="type")

display(hr_analysis.head())


################################################
display(Markdown("##### Statistics"))

steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "subphase",
    "subject": "subject",
    "groupby": ["type", "phase"],
    "multicomp": {"levels": "phase"}
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_hrv_response_bl_at.xlsx"))
stats.display_results(prep=False)


################################################
display(Markdown("##### Latex Output"))

index_rename_map = {
    "HR_Norm": "HR",
    "HRV_RMSSD": "RMSSD",
    "HRV_pNN50": "pRR50"
}

index_value_order = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]

caption = "HR(V) responses of the Control condition to the MIST. \
Paired t-tests were performed between BL and AT subphases for each individual MIST phase.\
"

df_latex = stats.results_to_latex_table(
    "pairwise_ttests", 
    unstack_levels="phase",
    index_kws={
        "index_value_order": index_value_order, 
        "index_rename_map": index_rename_map,
        "index_level_names_tex": ["Measure"]
    },
    caption=caption,
    label="tab:hrv_response_mist"
)

for path in [tex_path, paper_tex_path]:
    if path is not None:
        path.joinpath("tab_hrv_response_bl_at.tex").open(mode="w+").write(df_latex);
print(df_latex)

##### Prepare Data

data
condition subject phase subphase category type                
Control   Vp03    MIST1 AT       HR       HR_Norm     0.275049
                                 HRV      HRV_RMSSD  11.778790
                                          HRV_pNN50   0.242718
                        BL       HR       HR_Norm    -5.230397
                                 HRV      HRV_RMSSD  21.937769

##### Statistics

<font size="3"><b> Overview </b></font>

,dv,within,subject,groupby,multicomp
parameter,data,subphase,subject,"['type', 'phase']",{'levels': 'phase'}


...

<font size="3"><b> Statistical Tests </b></font>

**Pairwise t-Tests**

Contrast   A   B  Paired  Parametric       T   dof  \
type      phase                                                         
HRV_RMSSD MIST1 0  subphase  AT  BL    True        True -1.2899  12.0   
          MIST2 0  subphase  AT  BL    True        True -3.1026  12.0   
          MIST3 0  subphase  AT  BL    True        True -1.9357  12.0   
HRV_pNN50 MIST1 0  subphase  AT  BL    True        True -0.7235  12.0   
          MIST2 0  subphase  AT  BL    True        True -2.7398  12.0   
          MIST3 0  subphase  AT  BL    True        True -1.1772  12.0   
HR_Norm   MIST1 0  subphase  AT  BL    True        True  3.3113  12.0   
          MIST2 0  subphase  AT  BL    True        True  4.1992  12.0   
          MIST3 0  subphase  AT  BL    True        True  3.8499  12.0   

                  alternative   p-unc    BF10  hedges  p-corr  
type      phase                                                
HRV_RMSSD MIST1 0   two-sided  0.2214   0.552 -0.2024  0.6642  
          MIST2 0   two-sided  0.0091   6.231 -0.5338  0.0273  
          MIST3 0   two-sided  0.0768   1.166 -0.3515  0.2304  
HRV_pNN50 MIST1 0   two-sided  0.4833   0.349 -0.0518  1.0000  
          MIST2 0   two-sided  0.0179   3.612 -0.4735  0.0537  
          MIST3 0   two-sided  0.2619   0.495 -0.2578  0.7857  
HR_Norm   MIST1 0   two-sided  0.0062   8.564  0.6553  0.0186  
          MIST2 0   two-sided  0.0012  33.143  0.8936  0.0036  
          MIST3 0   two-sided  0.0023  19.512  0.7616  0.0069

##### Latex Output

\begin{table}[th!]
\centering
\caption{HR(V) responses of the Control condition to the MIST. Paired t-tests were performed between BL and AT subphases for each individual MIST phase.}
\label{tab:hrv_response_mist}
\sisetup{table-format = <1.3}

\begin{tabular}{l||SSS|SSS|SSS}
\toprule
{} & \multicolumn{3}{c}{{MIST1}} & \multicolumn{3}{c}{{MIST2}} & \multicolumn{3}{c}{{MIST3}} \\
{} & {$t(12)$} &          {p} & {Hedges' g} & {$t(12)$} &           {p} & {Hedges' g} & {$t(12)$} &           {p} & {Hedges' g} \\
\textit{Measure} &           &              &             &           &               &             &           &               &             \\
\midrule
\textit{HR}      &     3.311 &  0.019$^{*}$ &       0.655 &     4.199 &  0.004$^{**}$ &       0.894 &     3.850 &  0.007$^{**}$ &       0.762 \\
\textit{RMSSD}   &    -1.290 &   0.664$^{}$ &      -0.202 &    -3.103 &   0.027$^{*}$ &      -0.534 &    -1.936 &    0.230$^{}$ &      -0.351 \\
\textit{pRR50}   &    -0.724 &  >0.999$^{}$

#### Increase over MIST

In [10]:
display(Markdown("##### Prepare Data"))

measures = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]
subphases = ["BL", "AT", "FB"]
conditions = ["Control"]

hr_analysis = dataset.get_subset(condition=conditions, subphase=subphases).hr_hrv

# Select data
hr_analysis = multi_xs(hr_analysis, measures, level="type")
display(hr_analysis.head())

#################################################################
display(Markdown("##### Statistics"))

steps = [
    ("prep", "normality"),
    ("prep", "equal_var"),
    ("test", "rm_anova"),
    ("posthoc", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "phase",
    "subject": "subject",
    "groupby": ["subphase", "type"],
    "multicomp": {"levels": False}
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_hrv_response_mist_phases.xlsx"))
stats.display_results(prep=True)#, sig_only="posthoc")

#################################################################
display(Markdown("##### Latex Output"))
index_rename_map = {
    "HR_Norm": "HR",
    "HRV_RMSSD": "RMSSD",
    "HRV_pNN50": "pRR50"
}

index_value_order = {
    "type": ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"],
    "subphase": ["BL", "AT", "FB"]
}

caption = "HR(V) measures during MIST for Control condition. \
Repeated-measurement ANOVAs were performed separately for BL, AT, and FB subphases. \
Within-variable: MIST phase."

df_latex = stats.results_to_latex_table(
    "rm_anova",
    unstack_levels=["subphase"],
    index_kws={
        "index_value_order": index_value_order,
        "index_rename_map": index_rename_map,
        "index_level_names_tex": ["Measure"]
    },
    caption=caption,
    label="tab:hrv_increase_mist",
    multicolumn_format="c",
    escape=False,
    multirow=True,
)

for path in [tex_path, paper_tex_path]:
    if path is not None:
        paper_tex_path.joinpath("tab_hrv_response_mist_phases.tex").open(mode="w+").write(df_latex);

print(stats.stats_to_latex("pairwise_ttests"))
print(df_latex)

##### Prepare Data

data
condition subject phase subphase category type                
Control   Vp03    MIST1 AT       HR       HR_Norm     0.275049
                                 HRV      HRV_RMSSD  11.778790
                                          HRV_pNN50   0.242718
                        BL       HR       HR_Norm    -5.230397
                                 HRV      HRV_RMSSD  21.937769

##### Statistics

<font size="3"><b> Overview </b></font>

,dv,within,subject,groupby,multicomp
parameter,data,phase,subject,"['subphase', 'type']",{'levels': False}


...

<font size="3"><b> Post-Hoc Analysis </b></font>

**Pairwise t-Tests**

Contrast      A      B  Paired  Parametric       T   dof  \
subphase type                                                                   
AT       HRV_RMSSD 0    phase  MIST1  MIST2    True        True  2.7194  12.0   
                   1    phase  MIST1  MIST3    True        True  3.5877  12.0   
                   2    phase  MIST2  MIST3    True        True  1.3972  12.0   
         HRV_pNN50 0    phase  MIST1  MIST2    True        True  2.8786  12.0   
                   1    phase  MIST1  MIST3    True        True  3.3882  12.0   
                   2    phase  MIST2  MIST3    True        True  0.9337  12.0   
         HR_Norm   0    phase  MIST1  MIST2    True        True -2.8317  12.0   
                   1    phase  MIST1  MIST3    True        True -4.1871  12.0   
                   2    phase  MIST2  MIST3    True        True -1.5710  12.0   
BL       HRV_RMSSD 0    phase  MIST1  MIST2    True        True  0.3171  12.0   
                   1    phase  MIST1  MIST3    True        True  1.8607  12.0   
                   2    phase  MIST2  MIST3    True        True  1.7031  12.0   
         HRV_pNN50 0    phase  MIST1  MIST2    True        True  0.5274  12.0   
                   1    phase  MIST1  MIST3    True        True  1.7678  12.0   
                   2    phase  MIST2  MIST3    True        True  1.2447  12.0   
         HR_Norm   0    phase  MIST1  MIST2    True        True -1.3782  12.0   
                   1    phase  MIST1  MIST3    True        True -4.0193  12.0   
                   2    phase  MIST2  MIST3    True        True -2.3987  12.0   
FB       HRV_RMSSD 0    phase  MIST1  MIST2    True        True  0.8470  12.0   
                   1    phase  MIST1  MIST3    True        True  3.1000  12.0   
                   2    phase  MIST2  MIST3    True        True  3.2918  12.0   
         HRV_pNN50 0    phase  MIST1  MIST2    True        True  0.9734  12.0   
                   1    phase  MIST1  MIST3    True        True  1.4420  12.0   
                   2    phase  MIST2  MIST3    True        True  1.4801  12.0   
         HR_Norm   0    phase  MIST1  MIST2    True        True -1.9363  12.0   
                   1    phase  MIST1  MIST3    True        True -3.5281  12.0   
                   2    phase  MIST2  MIST3    True        True -2.7546  12.0   

                     alternative   p-unc    BF10  hedges  p-corr  
subphase type                                                     
AT       HRV_RMSSD 0   two-sided  0.0186   3.504  0.4796  0.0558  
                   1   two-sided  0.0037  13.071  0.6576  0.0111  
                   2   two-sided  0.1877   0.616  0.2044  0.5631  
         HRV_pNN50 0   two-sided  0.0139   4.443  0.5318  0.0417  
                   1   two-sided  0.0054   9.632  0.6618  0.0162  
                   2   two-sided  0.3689   0.403  0.1446  1.0000  
         HR_Norm   0   two-sided  0.0151   4.141 -0.8920  0.0453  
                   1   two-sided  0.0013  32.546 -1.3317  0.0039  
                   2   two-sided  0.1422   0.746 -0.4142  0.4266  
BL       HRV_RMSSD 0   two-sided  0.7566   0.291  0.0502  1.0000  
                   1   two-sided  0.0875   1.059  0.4388  0.2625  
                   2   two-sided  0.1143   0.871  0.3808  0.3429  
         HRV_pNN50 0   two-sided  0.6075   0.314  0.0902  1.0000  
                   1   two-sided  0.1025   0.943  0.3767  0.3075  
                   2   two-sided  0.2370   0.528  0.3011  0.7110  
         HR_Norm   0   two-sided  0.1933   0.604 -0.4111  0.5799  
                   1   two-sided  0.0017  25.248 -1.0276  0.0051  
                   2   two-sided  0.0336   2.196 -0.6432  0.1008  
FB       HRV_RMSSD 0   two-sided  0.4136   0.378  0.1045  1.0000  
                   1   two-sided  0.0092   6.206  0.5136  0.0276  
                   2   two-sided  0.0064   8.312  0.3832  0.0192  
         HRV_pNN50 0   two-sided  0.3496   0.415  0.1297  1.0000  
                   1   two-sided  0.1749   0.646  0.3004  0.524

##### Latex Output

subphase  type          Contrast  A      B    
AT        HRV_RMSSD  0  phase     MIST1  MIST2      $t(12) = 2.719, p = 0.056, g = 0.480$
                     1  phase     MIST1  MIST3      $t(12) = 3.588, p = 0.011, g = 0.658$
                     2  phase     MIST2  MIST3      $t(12) = 1.397, p = 0.563, g = 0.204$
          HRV_pNN50  0  phase     MIST1  MIST2      $t(12) = 2.879, p = 0.042, g = 0.532$
                     1  phase     MIST1  MIST3      $t(12) = 3.388, p = 0.016, g = 0.662$
                     2  phase     MIST2  MIST3      $t(12) = 0.934, p > 0.999, g = 0.145$
          HR_Norm    0  phase     MIST1  MIST2    $t(12) = -2.832, p = 0.045, g = -0.892$
                     1  phase     MIST1  MIST3    $t(12) = -4.187, p = 0.004, g = -1.332$
                     2  phase     MIST2  MIST3    $t(12) = -1.571, p = 0.427, g = -0.414$
BL        HRV_RMSSD  0  phase     MIST1  MIST2      $t(12) = 0.317, p > 0.999, g = 0.050$
                     1  phase     MIST1  MIST3      $

## Response to the CFT

In [11]:
features = ["onset_latency", "peak_brady_latency", "peak_brady_percent", "mean_brady_percent"]

### Descriptive Analysis

In [12]:
cft_analysis = dataset.cft_parameter
cft_analysis.head()

data
condition subject phase subphase category type                         
CFT       Vp01    MIST1 Total    CFT      baseline_hr         92.059566
                                          cft_start_idx       93.000000
                                          mean_brady_bpm      -7.348546
                                          mean_brady_percent  -7.982382
                                          mean_hr_bpm         84.711019

In [13]:
cft_analysis = multi_xs(cft_analysis, features, level="type")
cft_analysis_agg = cft_analysis.unstack()["data"]

cft_analysis_agg = cft_analysis_agg.groupby("phase").agg(
    {
        "onset_latency": ["mean", "std"], 
        "peak_brady_latency": ["mean", "std"], 
        "peak_brady_percent": ["min", "mean", "std"], 
        "mean_brady_percent": ["min", "mean", "std"]
    }
).T
cft_analysis_agg

phase                        MIST1      MIST2      MIST3
type                                                    
onset_latency      mean   7.525250   4.307167  12.229667
                   std    8.748095   4.555121   6.752471
peak_brady_latency mean  50.941250  47.746250  53.456500
                   std   29.898792  34.009036  32.137150
peak_brady_percent min  -26.617372 -23.891467 -20.651473
                   mean -18.773242 -16.589792 -12.261732
                   std    4.833408   4.170970   3.874516
mean_brady_percent min  -15.320881 -11.770874  -7.061865
                   mean  -6.192499  -3.939586  -0.836625
                   std    5.430712   4.655319   4.315025

In [16]:
cft_time_bl_glo = dataset.get_subset(condition="CFT", subphase="RP_CFI").time_above_baseline
cft_time_bl_glo = cft_time_bl_glo.xs("HR", level="type")
cft_time_bl_glo = cft_time_bl_glo.groupby("phase").agg(["mean", "std"]).T
cft_time_bl_glo.round(2)

phase      MIST1  MIST2  MIST3
data mean   9.39  10.20  20.12
     std   15.27   8.98  17.51

### Statistical Analysis

**Population**: CFT condition  
**Analysis** (per measure):
1. *Decrease BL-RP_CFI*: Check whether Cold Face Intervention causes significant HR(V) responses.
    * Procedure: Paired t-tests between subphases *BL* and *RP_CFI* for each MIST phase
    * Expected Result: Significant HR(V) responses during each MIST phase


**Findings**: 
* Each MIST Phase induces stress, indicated by significant differences between $BL_{Loc}$ and $AT$ in each individual MIST Phase
* Stress levels increase over time, indicated by significant main effect *MIST Phase* and significant differences between MIST phases, especially between *MIST1* and *MIST3*

#### Decrease *BL-RP_CFI*

In [ ]:
display(Markdown("##### Prepare Data"))

subphases = ["BL", "RP_CFI"]
conditions = ["CFT"]
measures = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]

hr_analysis = dataset.get_subset(condition=conditions, subphase=subphases).hr_hrv
hr_analysis = multi_xs(hr_analysis, measures, level="type")
display(hr_analysis.head())

################################################################
display(Markdown("##### Statistics"))

steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "subphase",
    "subject": "subject",
    "groupby": ["type", "phase"],
    "multicomp": {"levels": "phase"},
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_hr_response_bl_cfi.xlsx"))
stats.display_results(prep=False)

################################################################
display(Markdown("##### Latex Output"))

index_rename_map = {
    "HR_Norm": "HR",
    "HRV_RMSSD": "RMSSD",
    "HRV_pNN50": "pRR50"
}

index_value_order = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]

caption="HR(V) responses of the CFI condition to the Cold Face Test. \
Paired t-tests were performed between BL and RP/CFI subphases for each MIST phase separately."

df_latex = stats.results_to_latex_table(
    "pairwise_ttests", 
    unstack_levels="phase",
    index_kws={
        "index_value_order": index_value_order, 
        "index_rename_map": index_rename_map,
        "index_level_names_tex": ["Measure"]
    },
    caption=caption,
    label="tab:hrv_response_cft"
)

for path in [tex_path, paper_tex_path]:
    if path is not None:
        path.joinpath("tab_hrv_response_bl_cfi.tex").open(mode="w+").write(df_latex);
print(df_latex)

## Effect of CFT on Acute Stress Response

### Statistical Analysis

#### Interaction Condition x MIST

**Population**: Control vs. CFT condition  

**Analysis** (per measure):
1. *Interaction Condition x MIST Phase*: Check whether Cold Face Intervention has an effect on the HR(V) responses to the MIST.
    * Procedure:
        * Mixed-ANOVA to determine interaction effect between *Condition* and *MIST Phase*
        * In case of significant interaction effect: Post-hoc test to assess during which MIST Phase the conditions showed HR(V) response differences
    * Expected Result: Significant interaction effect between *Condition* and *MIST Phase*


**Findings**: 
* Each MIST Phase induces stress, indicated by significant differences between $BL_{Loc}$ and $AT$ in each individual MIST Phase
* Stress levels increase over time, indicated by significant main effect *MIST Phase* and significant differences between MIST phases, especially between *MIST1* and *MIST3*

In [ ]:
display(Markdown("##### Prepare Data"))

phases = ["MIST1", "MIST2", "MIST3"]
subphases = ["BL", "RP_CFI"]
hr_measures = ["HR_Norm", "HRV_RMSSD", "HRV_pNN50"]
time_bl_measures = ["HR", "HRV_RMSSD", "HRV_pNN50"]

hr_analysis = dataset.get_subset(phase=phases, subphase=subphases).hr_hrv
hr_analysis = multi_xs(hr_analysis, hr_measures, level="type")
time_bl_analysis = dataset.get_subset(phase=phases, subphase=subphases).time_above_baseline
time_bl_analysis = multi_xs(time_bl_analysis, time_bl_measures, level="type")

hr_analysis = hr_analysis.unstack(["type", "category"]).join(time_bl_analysis.unstack(["type", "category"]))
hr_analysis = hr_analysis.stack(["category", "type"])
display(hr_analysis.head())


#################################################################
display(Markdown("##### Statistics"))

steps = [
    ("prep", "normality"),
    ("prep", "equal_var"),
    ("test", "mixed_anova"),
    ("posthoc", "pairwise_ttests")
]
params = {
    "dv": "data",
    "within": "phase",
    "between": "condition",
    "subject": "subject",
    "groupby": ["category", "type", "subphase"],
    "padjust": "bonf"
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(hr_analysis)

stats.export_statistics(stats_path.joinpath("stats_effect_cft_interaction.xlsx"))
stats.display_results(prep=False, posthoc=False, sig_only=["test", "posthoc"])

#################################################################
display(Markdown("##### Latex Output"))

index_rename_map = {
    "HR_HR_Norm": "$\Delta HR$",
    "HRV_HRV_RMSSD": "RMSSD",
    "HRV_HRV_pNN50": "pRR50",
    "RP_CFI": "RP/CFI",
    "Time_BL_Glo_HR": "$\hat{t}_{Glo}(HR)$",
    "Time_BL_Glo_HRV_RMSSD": "$\hat{t}_{Glo}(RMSSD)$",
    "Time_BL_Glo_HRV_pNN50": "$\hat{t}_{Glo}(pRR50)$"
}

index_value_order = {
    "type": [
        "HR_HR_Norm", "Time_BL_Glo_HR", "HRV_HRV_RMSSD", "Time_BL_Glo_HRV_RMSSD", "HRV_HRV_pNN50", "Time_BL_Glo_HRV_pNN50"
    ]
}

#################################################################
display(Markdown("###### Interaction Effect"))

data = stats.results["mixed_anova"]
data = data.xs("BL", level="subphase")
data = data.unstack(-1)
data.index = ["_".join(val) for val in data.index]
data.index.names = ["type"]
data = data.stack(-1)

caption = "Mixed-ANOVA results (interaction effect MIST Phase x Condition) of HR(V) measures during BL subphase."

df_latex = stats.results_to_latex_table(
    "mixed_anova",
    data=data,
    stats_effect_type="Interaction",
    collapse_dof=True,
    index_kws={
        "index_value_order": index_value_order, 
        "index_rename_map": index_rename_map,
        "index_level_names_tex": ["Measure"]
    },
    caption=caption,
    label="tab:cft_mist_interaction"
)

for path in [tex_path, paper_tex_path]:
    if path is not None:
        path.joinpath("tab_hrv_response_cft_interaction.tex").open(mode="w+").write(df_latex);
print(df_latex)


#################################################################
display(Markdown("###### Main Effect Condition"))

data = stats.results["mixed_anova"]
data = data.xs("RP_CFI", level="subphase")
data = data.unstack(-1)
data.index = ["_".join(val) for val in data.index]
data.index.names = ["type"]
data = data.stack(-1)

caption = "Mixed-ANOVA results (main effect Condition) of HR(V) measures during RP/CFI subphase."

df_latex = stats.results_to_latex_table(
    "mixed_anova",
    data=data,
    stats_effect_type="condition",
    index_kws={
        "index_value_order": index_value_order, 
        "index_rename_map": index_rename_map,
        "index_level_names_tex": ["Measure"]
    },
    caption=caption,
    label="tab:cft_mist_main_effect"
)
for path in [tex_path, paper_tex_path]:
    if path is not None:
        path.joinpath("tab_hrv_response_cft_main_effect.tex").open(mode="w+").write(df_latex);
print(df_latex)

#################################################################
display(Markdown("###### Post-Hoc"))

posthoc_results = stats._filter_effect("posthoc", "interaction")
posthoc_results = multi_xs(posthoc_results, ["HR", "Time_BL_Glo"], level="category")
posthoc_results = multi_xs(posthoc_results, ["HR", "HR_Norm"], level="type")
posthoc_results = posthoc_results.xs("BL", level="subphase")
posthoc_results = posthoc_results.set_index("phase", append=True)

print(stats.stats_to_latex(data=posthoc_results))

### Plots

In [ ]:
category = ["HR"]
feature = ["HR_Norm"]
subphase = ["BL"]

hr_plot = dataset.get_subset(subphase=subphase).heart_rate
hr_plot = multi_xs(hr_plot, feature, level="type")

stats_data = stats.results_cat("posthoc").loc[category, feature, subphase]
box_pairs, pvalues = stats.sig_brackets(stats_data, stats_effect_type="interaction", plot_type="multi", x="phase")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=hr_plot.reset_index(), 
    x="phase", 
    y="data", 
    hue="condition", 
    hue_order=hue_order, 
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues},
    legend_loc="upper left",
    ax=ax
)

ax.set_xlabel("Phase")
ax.set_ylabel("$\Delta$ HR during $BL_{Loc}$ [%]")

fig.tight_layout()

In [ ]:
category = ["Time_BL_Glo"]
feature = ["HR"]
subphase = ["BL"]

hr_plot = dataset.get_subset(subphase=subphase).time_above_baseline
hr_plot = multi_xs(hr_plot, feature, level="type")


stats_data = stats.results_cat("posthoc").loc[category, feature, subphase, :]
box_pairs, pvalues = stats.sig_brackets(stats_data, stats_effect_type="interaction", plot_type="multi", x="phase")

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=hr_plot.reset_index(), 
    x="phase", 
    y="data", 
    hue="condition", 
    hue_order=hue_order, 
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues},
    legend_loc="upper left",
    ax=ax
)


ax.set_xlabel("Phase")
ax.set_ylabel("$\hat{t}_{Glo}(HR)$ during BL [%]")

fig.tight_layout()